In [4]:
# pip install youtube-dl


In [19]:

import os
import cv2
import torch 
import numpy as np       
import pandas as pd       
import matplotlib.pyplot as plt  
from pytube import YouTube  


### Loading the Video

In [20]:

# Load and check the video
video_path = 'Video data.mp4'  
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Video not Downloaded.")
else:
    print("Video Loaded successfully!")


Video Loaded successfully!


### Initial Background Subtraction and Contour Detection

In [25]:

# Initializing background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=True)
cap = cv2.VideoCapture(video_path)

# morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # background subtraction
    fg_mask = bg_subtractor.apply(frame)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # Displaying the frame with detections
    cv2.imshow('Car Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


### Save Detected Frames

In [ ]:

# Create a directory to save detected frames
output_dir = 'detected_frames'
os.makedirs(output_dir, exist_ok=True)

bg_subtractor = cv2.createBackgroundSubtractorMOG2()
cap = cv2.VideoCapture(video_path)
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Applying  background subtraction
    fg_mask = bg_subtractor.apply(frame)
    # Find contours in the mask
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Save the frame with detection
            frame_output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')  # Padded frame number
            cv2.imwrite(frame_output_path, frame)
            frame_count += 1

    # Displaying the frame with detections
    cv2.imshow('Car Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


### Object Tracking with KCF Tracker

In [ ]:

bg_subtractor = cv2.createBackgroundSubtractorMOG2()
cap = cv2.VideoCapture('video_path')

# List to store trackers
trackers = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_trackers = []

    for contour in contours:
        if cv2.contourArea(contour) > 500:  
            x, y, w, h = cv2.boundingRect(contour)
            tracker = cv2.TrackerKCF_create()  
            tracker.init(frame, (x, y, w, h))
            new_trackers.append(tracker)

    # Update all trackers and draw rectangles
    for tracker in trackers:
        success, box = tracker.update(frame)
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Update the list of trackers
    trackers = new_trackers

    # Display the frame with detections and tracking
    cv2.imshow('Car Detection and Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### Counting the Cars

In [ ]:

bg_subtractor = cv2.createBackgroundSubtractorMOG2()
cap = cv2.VideoCapture('video_path')

# Initialize tracker
tracker = cv2.TrackerKCF_create()
# List to store tracked objects
tracked_objects = []
car_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_objects = []

    for contour in contours:
        if cv2.contourArea(contour) > 500: 
            x, y, w, h = cv2.boundingRect(contour)
            new_objects.append((x, y, w, h))

            # Initialize a new tracker for new objects
            tracker = cv2.TrackerKCF_create()
            tracker.init(frame, (x, y, w, h))

    # Update all trackers and count unique cars
    for obj in tracked_objects:
        success, box = obj[0].update(frame)
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            # Remove lost trackers
            tracked_objects.remove(obj)

    # Add new objects to the tracked list
    for obj in new_objects:
        tracker = cv2.TrackerKCF_create()
        tracker.init(frame, obj)
        tracked_objects.append((tracker, obj))
        
    cv2.imshow('Car Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f'Total number of unique cars detected: {len(tracked_objects)}')


### Optimized Background Subtraction and Contour Detection

In [ ]:

# Initialize background subtractor with optimized parameters
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=True)

cap = cv2.VideoCapture('video_path')

# Define minimum contour area for filtering noise
min_contour_area = 500

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:  # Filter small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Displaying the frame with detections
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
max_contour_area = 5000  # Example value
if min_contour_area < cv2.contourArea(contour) < max_contour_area:
    # Proceed with contour processing


In [ ]:
max_contour_area = 5000  # Example value
if min_contour_area < cv2.contourArea(contour) < max_contour_area:
    # Proceed with contour processing..........................................
